In [13]:
import lammps_logfile
import scipy
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg, NavigationToolbar2Tk
import PySimpleGUI as sg
import numpy as np
import PySimpleGUI as sg
import matplotlib.widgets as wg
import matplotlib
import matplotlib.pyplot as plt
import import_ipynb
from ase.visualize.plot import plot_atoms
from ase import Atoms
import math
from methods import *

sg.theme("DarkBlue7")

plt.style.use('dark_background')
color = sg.theme_background_color()

plotting_fit_data = {}
plotting_rdf_data = {}
plotting_view_data = {}
selected_rows_dict = {}

In [26]:
import colorsys


def adjust_lightness(color, amount=0.5):
    import matplotlib.colors as mc
    import colorsys
    try:
        c = mc.cnames[color]
    except:
        c = color
    c = colorsys.rgb_to_hls(*mc.to_rgb(c))
    x = colorsys.hls_to_rgb(c[0], max(0, min(1, amount * c[1])), c[2])
    ret = [int(l * 255) for l in x]
    return '#{:02X}{:02X}{:02X}'.format(ret[0], ret[1], ret[2])


color2 = adjust_lightness(color, 3)
# color2='#241663'
color2

'#110A31'

In [7]:
# Method used in all of the plotting functions. It allows for integration of matplotlib's
# plots and widgets with PYSimpleGUI. As parameters takes figure's canvas, toolbar's canvas
# and matplotlib figure to be drawn.
def draw_figure_w_toolbar(canvas, fig, canvas_toolbar):
    if canvas.children:
        for child in canvas.winfo_children():
            child.destroy()
    if canvas_toolbar.children:
        for child in canvas_toolbar.winfo_children():
            child.destroy()
    figure_canvas_agg = FigureCanvasTkAgg(fig, master=canvas)
    figure_canvas_agg.draw()
    toolbar = Toolbar(figure_canvas_agg, canvas_toolbar)
    for button in toolbar.winfo_children():  # Important for color theme
        button.config(background=color2)
    toolbar.update()
    figure_canvas_agg.get_tk_widget().pack(side='right', fill='both', expand=1)

In [3]:
class Toolbar(NavigationToolbar2Tk):
    def __init__(self, *args, **kwargs):
        super(Toolbar, self).__init__(*args, **kwargs)
        self.config(background=color2)  # Important for color theme

In [1]:
# Method used for plotting four plots in main window's tabs. As parameters takes log and
# unit_types data, canvases of a figure and toolbar, values dictionary that contains
# values of all elements in the window, tab_id which is absolute path of a file from
# which the plotted data is retrieved.
def plot_main(log, unit_types, fig_cv, toolbar_cv, values, tab_id):
    n, xq, yq, ut, x, y = ([] for i in range(6))

    for i in range(4):
        n.append(values[f'combo_n{i+1}_{tab_id}'] - 1)
        xq.append(values[f'combo_x{i+1}_{tab_id}'])
        yq.append(values[f'combo_y{i+1}_{tab_id}'])
        ut.append(unit_types[n[i]])
        x.append(log.get(xq[i], n[i]))
        y.append(log.get(yq[i], n[i]))

    fig, axs = plt.subplots(2, 2)
    fig.set_facecolor(color)
    def a(x): return 1 if x > 1 else 0
    def b(x): return 0 if x % 2 == 0 else 1
    for i in range(4):
        axs[a(i), b(i)].scatter(x[i], y[i])
        axs[a(i), b(i)].set(xlabel=xq[i] + ' (' + units(ut[i], xq[i]) + ')',
                            ylabel=yq[i] + ' (' + units(ut[i], yq[i]) + ')')

    fig.tight_layout()

    plt.figure(1)
    fig = plt.gcf()
    plt.close()
    DPI = fig.get_dpi()
    # Play with this size to reduce the movement error
    # when the mouse hovers over the figure, it's close to canvas size.
    fig.set_size_inches(500 * 2 / float(DPI), 500 / float(DPI))
    draw_figure_w_toolbar(fig_cv, fig, toolbar_cv)

In [5]:
def plot_rdf(log, unit_types, fig_cv, toolbar_cv, values, win_id, win):
    plotting_rdf_data[win_id] = {}
    plotting_rdf_data[win_id]['point1'] = None
    plotting_rdf_data[win_id]['point2'] = None
    plotting_rdf_data[win_id]['x'] = np.array([])
    plotting_rdf_data[win_id]['y'] = np.array([])

    n = values[f'combo_nrdf_{win_id}'] - 1
    xq = values[f'combo_xrdf_{win_id}']
    yq = values[f'combo_yrdf_{win_id}']

    ut = unit_types[n]
    x = log.get(xq, n)
    y = log.get(yq, n)
    ux = units(ut, xq)
    uy = units(ut, yq)

    fig, axs = plt.subplots(1, 2)
    fig.set_facecolor(color)
    axs[0].scatter(x, y)
    axs[0].set(xlabel=xq + " (" + ux + ")",
               ylabel=yq + " (" + uy + ")")
    axs[1].set(xlabel='Distance ($\\AA$)', ylabel='RDF')

    fig.tight_layout()
    plt.figure(1)
    fig = plt.gcf()
    plt.close()
    DPI = fig.get_dpi()
    # you have to play with this size to reduce the movement error
    # when the mouse hovers over the figure, it's close to canvas size
    fig.set_size_inches(500 * 2 / float(DPI), 500 / float(DPI))
    draw_figure_w_toolbar(fig_cv, fig, toolbar_cv)

    def on_click_rdf(event):
        x0 = event.xdata
        y0 = event.ydata
        if event.inaxes is axs[0]:
            point1 = plotting_rdf_data[win_id]['point1']
            if point1 is not None:
                point = point1.pop(0)
                point.remove()
                plotting_rdf_data[win_id]['point1'] = None
                fig.canvas.draw()

            x_cp, y_cp = find_closest_point(x, y, x0, y0)
            if win_id in list(rdf_dict.keys()):
                index = np.where(x == x_cp)[0][0]
                step = log.get('Step', n)
                key = step[index]
                key_list = list(rdf_dict[win_id].keys())
                key_list = [int(i) for i in key_list]
                key_list = np.array(key_list)
                key = find_nearest_value(key_list, key)
                new_index = np.where(step == key)[0]
                rdf_data = np.array(rdf_dict[win_id][str(key)])

                x_cp = x[new_index]
                y_cp = y[new_index]
                rdf_col_number = 1
                if values[f'combo_colrdf_{win_id}'] != '':
                    rdf_col_number = int(values[f'combo_colrdf_{win_id}'])

                x_rdf = rdf_data[:, 1]
                y_rdf = rdf_data[:, rdf_col_number+1]
                plotting_rdf_data[win_id]['x'] = x_rdf
                plotting_rdf_data[win_id]['y'] = y_rdf
                axs[1].cla()
                axs[1].plot(x_rdf, y_rdf)
                axs[1].set(xlabel='Distance ($\\AA$)', ylabel='RDF')
                plotting_rdf_data[win_id]['point1'] = axs[0].plot(
                    x_cp, y_cp, color='red', marker='o')
                fig.canvas.draw()

        elif event.inaxes is axs[1] and np.size(plotting_rdf_data[win_id]['x']) and np.size(plotting_rdf_data[win_id]['y']):
            x_rdf = plotting_rdf_data[win_id]['x']
            y_rdf = plotting_rdf_data[win_id]['y']
            point2 = plotting_rdf_data[win_id]['point2']

            if win_id in list(rdf_dict.keys()):
                x_rdf_cp, y_rdf_cp = find_closest_point(
                    x_rdf, y_rdf, x0, y0)
                win[f'rdf_xy_{win_id}'].update(
                    f'X={format_number(x_rdf_cp)} Y={format_number(y_rdf_cp)}')

                if point2 is not None:
                    point = point2.pop(0)
                    point.remove()
                    plotting_rdf_data[win_id]['point2'] = None

                plotting_rdf_data[win_id]['point2'] = axs[1].plot(
                    x_rdf_cp, y_rdf_cp, color='red', marker='o')
                fig.canvas.draw()

    fig.canvas.mpl_connect("button_press_event", on_click_rdf)
    return axs

In [6]:
def plot_fit(log, unit_types, fig_cv, toolbar_cv, values, win_id, win):
    selected_rows_dict[win_id] = []
    plotting_fit_data[win_id] = {}
    plotting_fit_data[win_id]['boundries'] = []
    plotting_fit_data[win_id]['lines'] = []
    plotting_fit_data[win_id]['table_rows'] = []
    plotting_fit_data[win_id]['intersection_line'] = None

    win[f'table_fit_{win_id}'].update(values=[])

    n = values[f'combo_nfit_{win_id}'] - 1
    xq = values[f'combo_xfit_{win_id}']
    yq = values[f'combo_yfit_{win_id}']
    ut = unit_types[n]
    x = log.get(xq, n)
    y = log.get(yq, n)
    ux = units(ut, xq)
    uy = units(ut, yq)

    if ux == '' and uy == '':
        slope_unit = ''
    elif ux == '':
        slope_unit = f'(1/{uy})'
    elif uy == '':
        slope_unit = ux
    else:
        slope_unit = f'({uy}/{ux})'

    def single_unit(u):
        return f'({u})' if u != '' else ''

    new_headings = ['Line', f'Slope {slope_unit}',
                    f'Intercept {single_unit(uy)}', f'Boundry 1 {single_unit(ux)}', f'Boundry 2 {single_unit(ux)}']
    table = win[f'table_fit_{win_id}'].Widget
    update_headings(table, new_headings)

    fig, axs = plt.subplots(1)
    plotting_fit_data[win_id]['fit_axs'] = axs
    plotting_fit_data[win_id]['fit_fig'] = fig
    axs.scatter(x, y)
    axs.set(xlabel=xq + " (" + ux + ")",
            ylabel=yq + " (" + uy + ")")

    fig.set_facecolor(color)
    fig.tight_layout()

    plt.figure(1)
    fig = plt.gcf()
    plt.close()
    DPI = fig.get_dpi()
    # you have to play with this size to reduce the movement error
    # when the mouse hovers over the figure, it's close to canvas size
    fig.set_size_inches(500 * 2 / float(DPI), 500 / float(DPI))
    draw_figure_w_toolbar(fig_cv, fig, toolbar_cv)

    def on_click_fit(event):
        plotting_fit_data[win_id]['boundries'].append(event.xdata)
        fit_boundries = plotting_fit_data[win_id]['boundries']

        if len(fit_boundries) == 2:
            boundry1_index = np.where(x == min(
                x, key=lambda z: abs(fit_boundries[0] - z)))[0][0]
            boundry2_index = np.where(x == min(
                x, key=lambda z: abs(fit_boundries[1] - z)))[0][0]
            a, b = linear_fit(
                x, y, boundry1_index, boundry2_index)

            x_max, x_min = x.max(), x.min()
            x_line = np.arange(x_min, x_max, (x_max - x_min)/1000)
            label = len(plotting_fit_data[win_id]["lines"]) + 1
            line,  = axs.plot(x_line, a*x_line+b, label=f'{label}')
            axs.legend()
            fig.canvas.draw()
            plotting_fit_data[win_id]['lines'].append(line)

            plotting_fit_data[win_id]['table_rows'].append(
                [str(label), format_number(a), format_number(b), format_number(x[boundry1_index]), format_number(x[boundry2_index])])
            fit_table_rows = plotting_fit_data[win_id]['table_rows']
            win[f'table_fit_{win_id}'].update(fit_table_rows)

            number_of_rows = len(fit_table_rows)
            if number_of_rows == 1:
                new_selected_rows = [0]
                selected_rows_dict[win_id] = new_selected_rows
                win[f'table_fit_{win_id}'].update(
                    select_rows=new_selected_rows)
            else:
                new_selected_rows = [number_of_rows - 2, number_of_rows - 1]
                selected_rows_dict[win_id] = new_selected_rows
                win[f'table_fit_{win_id}'].update(
                    select_rows=new_selected_rows)

            plotting_fit_data[win_id]['boundries'].clear()

    fig.canvas.mpl_connect("button_press_event", on_click_fit)

    return axs

In [1]:
def plot_view(log, unit_types, fig_cv, toolbar_cv, values, win_id, win):
    plotting_view_data[win_id] = {}
    plotting_view_data[win_id]['point1'] = None

    n = values[f'combo_nview_{win_id}'] - 1
    xq = values[f'combo_xview_{win_id}']
    yq = values[f'combo_yview_{win_id}']
    ut = unit_types[n]
    x = log.get(xq, n)
    y = log.get(yq, n)
    ux = units(ut, xq)
    uy = units(ut, yq)

    fig, axs = plt.subplots(1, 2)
    axs[0].cla()
    axs[0].scatter(x, y)
    axs[0].set(xlabel=xq + " (" + ux + ")",
               ylabel=yq + " (" + uy + ")")

    axs[1].set_axis_off()
    fig.set_facecolor(color)
    fig.tight_layout()

    plt.figure(1)
    fig = plt.gcf()
    plt.close()
    DPI = fig.get_dpi()
    # you have to play with this size to reduce the movement error
    # when the mouse hovers over the figure, it's close to canvas size
    fig.set_size_inches(500 * 2 / float(DPI), 500 / float(DPI))
    draw_figure_w_toolbar(fig_cv, fig, toolbar_cv)

    def on_click_view(event):
        x0 = event.xdata
        y0 = event.ydata
        view_list = list(view_dict.keys())
        if event.inaxes is axs[0] and win_id in view_list:
            point1 = plotting_view_data[win_id]['point1']
            if point1 is not None:
                point = point1.pop(0)
                point.remove()
                plotting_view_data[win_id]['point1'] = None
                fig.canvas.draw()
            x_cp, y_cp = find_closest_point(x, y, x0, y0)
            index = np.where(x == x_cp)[0][0]
            step = log.get('Step', n)
            key = step[index]
            key_list = list(view_dict[win_id].keys())
            key_list = [int(i) for i in key_list]
            key_list = np.array(key_list)
            key = find_nearest_value(key_list, key)
            new_index = np.where(step == key)[0]

            x_cp = x[new_index]
            y_cp = y[new_index]

            atoms = view_dict[win_id][str(key)]

            if len(atoms) > 1000:
                distance_cutoff = 5.0
                center_of_mass = atoms.get_center_of_mass()
                new_atom_positions = []
                for i in range(len(atoms)):
                    if math.dist(atoms.positions[i], center_of_mass) < distance_cutoff:
                        new_atom_positions.append(atoms.positions[i] - center_of_mass + [5, 5, 5])
                atoms = Atoms(
                    positions=new_atom_positions, cell=[10, 10, 10])
            axs[1].cla()
            axs[1].set_axis_off()
            plot_atoms(atoms, axs[1], radii=1.0, rotation=('20x,30y,10z'))
            plotting_view_data[win_id]['point1'] = axs[0].plot(
                x_cp, y_cp, color='red', marker='o')
            fig.canvas.draw()

    fig.canvas.mpl_connect("button_press_event", on_click_view)

    return axs